In [2]:
import argparse
# from dataclasses import dataclass
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from dotenv import load_dotenv
load_dotenv()

CHROMA_PATH = "chroma"

PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

In [3]:
def main(query_text_local: str = None):
    if query_text_local:
        query_text = query_text_local
    else:
        # Create CLI.
        parser = argparse.ArgumentParser()
        parser.add_argument("query_text", type=str, help="The query text.")
        args = parser.parse_args()
        query_text = args.query_text

    # Prepare the DB.
    embedding_function = OpenAIEmbeddings()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)
    print("DB loaded.", db)

    # Search the DB.
    results = db.similarity_search_with_relevance_scores(query_text, k=3)
    print(f"Results: {results}")
    if len(results) == 0 or results[0][1] < 0.7:
        print(f"Unable to find matching results.")
        return

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    print(prompt)

    model = ChatOpenAI()
    response_text = model.predict(prompt)

    sources = [doc.metadata.get("source", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)


if __name__ == "__main__":
    main('Who is Harry Potter?')

/var/folders/73/ffdb84g14b37t5g6877s9nfr0000gn/T/ipykernel_34068/978532144.py:13: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)


DB loaded. <langchain_community.vectorstores.chroma.Chroma object at 0x11a9c2a50>
Results: [(Document(metadata={'source': 'data/books/Harry Potter 4 - The Goblet Of Fire.txt', 'start_index': 920648}, page_content='"What?" said Harry. "Rita Skeeter again?"\n\n"No," said Ron, and just like Hermione, he attempted to push the paper out of sight.\n\n"It\'s about me, isn\'t it?" said Harry.\n\n"No," said Ron, in an entirely unconvincing tone. But before Harry could demand to see the paper. Draco Malfoy shouted across the Great Hall from the Slytherin table.\n\n"Hey, Potter! Potter! How\'s your head? You feeling all right? Sure you\'re not going to go berserk on us?"\n\nMalfoy was holding a copy of the Daily Prophet too. Slytherins up and down the table were sniggering, twisting in their seats to see Harry\'s reaction.\n\n"Let me see it," Harry said to Ron. "Give it here."\n\nVery reluctantly, Ron handed over the newspaper. Harry turned it over and found himself staring at his own picture, be

/var/folders/73/ffdb84g14b37t5g6877s9nfr0000gn/T/ipykernel_34068/978532144.py:29: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response_text = model.predict(prompt)


Response: Harry Potter is the protagonist of the Harry Potter series, a wizard who attends Hogwarts School of Witchcraft and Wizardry. He is known for defeating the dark wizard Voldemort as a baby, has a lightning-shaped scar on his forehead as a result of the attack, and is often the subject of media scrutiny and rumors. He is portrayed as brave, loyal, and caring throughout the series.
Sources: ['data/books/Harry Potter 4 - The Goblet Of Fire.txt', "data/books/Harry Potter 1 - Sorcerer's Stone.txt", 'data/books/Harry Potter 5 - Order of the Phoenix.txt']
